# Dask DataFrame and cuDF on NYC Taxi CSV data

In [5]:
import dask.dataframe as dd
#import cudf
#import dask_cudf as cudd

from dask import persist, delayed
from dask.distributed import Client, progress, wait

### Start Dask Cluster on an Eight-GPU DGX Machine

In [15]:
#from dask_cuda import LocalCUDACluster
#cluster = LocalCUDACluster()

cluster_url = "172.17.0.2:8786"

client = Client(n_workers=4, threads_per_worker=1)
#client = Client(cluster)
client

/opt/conda/envs/rapids/lib/python3.8/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33955 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:33955/status,
Dashboard: http://127.0.0.1:33955/status,Workers: 4
Total threads: 4,Total memory: 62.82 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39727,Workers: 4
Dashboard: http://127.0.0.1:33955/status,Total threads: 4
Started: Just now,Total memory: 62.82 GiB
Comm: tcp://127.0.0.1:46029,Total threads: 1
Dashboard: http://127.0.0.1:37007/status,Memory: 15.70 GiB
Nanny: tcp://127.0.0.1:44057,


In [22]:
client.close()

In [16]:
nyc_datatype = {'VendorID': 'string',
                'passenger_count': 'int32',
                'trip_distance': 'float32',
                'pickup_longitude': 'float32',
                'pickup_latitude': 'float32',
                'RateCodeID': 'string',
                'store_and_fwd_flag': 'string',
                'dropoff_longitude': 'float32',
                'dropoff_latitude': 'float32',
                'payment_type': 'string',
                'fare_amount': 'float32',
                'extra': 'float32',
                'mta_tax': 'float32',
                'tip_amount': 'float32',
                'tolls_amount': 'float32',
                'improvement_surcharge': 'float32',
                'total_amount':'float32' }

### Previously we ran this to shard the files more finely for dask_cudf.read_csv

```python
import dask.dataframe as dd
pdf = dd.read_csv('data/nyc/yellow_tripdata_2017-*.csv',
                 parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

pdf.repartition(npartitions=100).to_csv('data/nyc/many/*.csv', index=False)
```

In [17]:
import dask.dataframe as dd

df = dd.read_parquet("/home/cloud/dataset/nyc-taxi/yellow_tripdata_2015.parquet")

df = df.astype(nyc_datatype)
df = df.persist()
progress(df)

VBox()

In [18]:
%time wait(df)
%time print(df.passenger_count.sum().compute())

CPU times: user 7.31 ms, sys: 116 µs, total: 7.43 ms
Wall time: 6.63 ms
245566747
CPU times: user 245 ms, sys: 40.3 ms, total: 285 ms
Wall time: 2.94 s


In [19]:
%time len(df)
%time df.passenger_count.sum().compute()

df.head()

CPU times: user 122 ms, sys: 3.23 ms, total: 126 ms
Wall time: 172 ms
CPU times: user 149 ms, sys: 9.72 ms, total: 159 ms
Wall time: 238 ms


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.049999
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.799999
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.800000
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.800000
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.299999


In [20]:
cols=df.columns
print(cols)
df[[cols[0], cols[1], cols[2]]].compute()
#df.get_partition(0).head().to_pandas()


Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58
...,...,...,...
409346,2,2015-12-31 23:59:56,2016-01-01 00:08:18
409347,1,2015-12-31 23:59:58,2016-01-01 00:05:19
409348,1,2015-12-31 23:59:59,2016-01-01 00:12:55
409349,2,2015-12-31 23:59:59,2016-01-01 00:10:26


In [21]:
#df.divisions
df.partitions[10].compute()
#df.passenger_count
#df["passenger_count"].compute()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-09 15:24:54,2015-01-09 15:26:05,1,0.04,-73.990318,40.761829,1,N,-73.990707,40.761795,2,3.0,0.0,0.5,0.0,0.0,0.3,3.800000
1,2,2015-01-09 15:24:54,2015-01-09 15:39:47,1,1.80,-73.969551,40.759357,1,N,-73.990929,40.750622,2,10.5,0.0,0.5,0.0,0.0,0.3,11.300000
2,2,2015-01-09 15:24:55,2015-01-09 15:39:30,1,1.46,-73.965836,40.762661,1,N,-73.984421,40.758846,2,10.5,0.0,0.5,0.0,0.0,0.3,11.300000
3,2,2015-01-09 15:24:55,2015-01-09 15:38:03,1,2.84,-73.988274,40.718346,1,N,-73.944115,40.704391,1,12.0,0.0,0.5,0.0,0.0,0.3,12.800000
4,2,2015-01-09 15:24:56,2015-01-09 15:40:16,3,2.03,-73.981575,40.778481,1,N,-73.954964,40.777527,2,11.5,0.0,0.5,0.0,0.0,0.3,12.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411192,2,2015-01-07 17:12:59,2015-01-07 17:29:51,1,1.86,-73.952080,40.777199,1,N,-73.970863,40.758671,2,11.5,1.0,0.5,0.0,0.0,0.3,13.300000
411193,2,2015-01-07 17:12:59,2015-01-07 17:45:20,1,4.67,-73.967186,40.793610,1,N,-73.986626,40.740646,1,22.5,1.0,0.5,2.7,0.0,0.3,27.000000
411194,2,2015-01-07 17:12:59,2015-01-07 17:40:13,1,5.05,-73.991379,40.755280,1,N,-73.945801,40.800411,2,20.5,1.0,0.5,0.0,0.0,0.3,22.299999
411195,2,2015-01-07 17:12:59,2015-01-07 17:22:56,1,1.25,-73.971359,40.760906,1,N,-73.975288,40.746319,2,8.0,1.0,0.5,0.0,0.0,0.3,9.800000


In [8]:
filepath = "/home/cloud/dataset/nyc-taxi/partition/yellow_tripdata_2015/*.csv"

#df.repartition(npartitions=180).to_csv(filepath, overwrite=True, index=False)
df.to_csv(filepath, overwrite=True, index=False)


TypeError: to_csv() got an unexpected keyword argument 'overwrite'

### Read Parquet files into Dask-GPU-DataFrame

In [2]:
import dask_cudf
gdf = dask_cudf.read_parquet(filepath)

%time gdf.head().to_pandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,1969-12-31 23:59:59,1969-12-31 23:59:59,1,3.30,1,-2049400382,263,161,1,12.5,0.0,0.5,2.00,0.0,0.3,15.30
1,1,1969-12-31 23:59:59,1969-12-31 23:59:59,1,0.90,1,-2049400382,186,234,1,5.0,0.0,0.5,1.45,0.0,0.3,7.25
2,1,1969-12-31 23:59:59,1969-12-31 23:59:59,1,1.10,1,-2049400382,164,161,1,5.5,0.0,0.5,1.00,0.0,0.3,7.30
3,1,1969-12-31 23:59:59,1969-12-31 23:59:59,1,1.10,1,-2049400382,236,75,1,6.0,0.0,0.5,1.70,0.0,0.3,8.50
4,2,1969-12-31 23:59:59,1969-12-31 23:59:59,1,0.02,2,-2049400382,249,234,2,52.0,0.0,0.5,0.00,0.0,0.3,52.80


### Time a full-pass computation

Most of the time here is spent reading data from disk and parsing it.

In [4]:
%time gdf.passenger_count.sum().compute()

CPU times: user 1.16 s, sys: 100 ms, total: 1.26 s
Wall time: 4.68 s


184464740

### Single GPU

In [6]:
%time gdf.passenger_count.sum().compute(scheduler='single-threaded')

CPU times: user 7.5 s, sys: 4.66 s, total: 12.2 s
Wall time: 10.9 s


184464740

### Single CPU

In [7]:
import dask.dataframe as dd

df = dd.read_csv('data/nyc/many/*.csv')
type(df.head())

pandas.core.frame.DataFrame

In [8]:
%time df.passenger_count.sum().compute(scheduler='single-threaded')

CPU times: user 34min 9s, sys: 21.5 s, total: 34min 30s
Wall time: 3min 14s


184464740

### Eight CPUs, one per process

In [9]:
%time df.passenger_count.sum().compute()

CPU times: user 10.8 s, sys: 988 ms, total: 11.8 s
Wall time: 57.5 s


184464740

### Eighty CPUs with a balance of threads and processes

In [10]:
client.close()
cluster.close()

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:49958 remote=tcp://127.0.0.1:33058>


In [11]:
client = Client(n_workers=10, threads_per_worker=8)
client

Client Scheduler: tcp://127.0.0.1:45873 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 10 Cores: 80 Memory: 540.96 GB


In [12]:
%time df.passenger_count.sum().compute()

CPU times: user 7.55 s, sys: 692 ms, total: 8.24 s
Wall time: 34.9 s


184464740